In [ ]:
import numpy as np
import pandas as pd

d1 = pd.read_csv('../temp/chdispositions_relabel.csv', header=0).set_index('tic')
d2 = pd.read_csv('../temp/chdispositions_uniq.csv', header=0).set_index('tic')

d3 = pd.concat([d1, d2])
d3 = d3.drop(columns=['Unnamed: 3'])

d4 = pd.read_csv('../temp/CHrelabel_part1.ls', names=['tic', 'nflag']).set_index('tic')
joined = d3.join(d4)
joined['flag'] = np.where(joined['nflag'].isna(), joined['flag'], joined['nflag'])
joined = joined[['user', 'flag']]

# joined.to_csv('../chdispositions.csv')

In [ ]:
def load_tces_old():
    tceold = pd.read_csv('astronet/tces.csv', header=0).set_index('tic_id')

    # Only keep the max sectors read.
    maxsect = tceold.groupby('tic_id')['Sectors'].max()
    tceold = tceold.join(maxsect, on='tic_id', how='right', rsuffix='_max')
    tceold = tceold[tceold.Sectors == tceold.Sectors_max]

    # Then keep the max row ID.
    maxrowid = tceold.groupby('tic_id')['row_id'].max()
    tceold = tceold.join(maxrowid, on='tic_id', how='right', rsuffix='_max')
    tceold = tceold[tceold.row_id == tceold.row_id_max]

    return tceold

In [ ]:
dliang = load_tces_old().reset_index()
dliang = dliang[dliang.Disposition == 'PC']
dliang = dliang.rename(columns={'tic_id': 'tic'})
dliang['user'] = 2
dliang['flag'] = 'E'

dliang = dliang[['user', 'flag', 'tic']].set_index('tic')

# dliang.to_csv('../yldispositions.csv')

In [ ]:
tcenew = pd.read_csv('../tce_bls_instar.csv', header=0).set_index('tic_id')
tceold = load_tces_old()

# Copy from old data where it's missing from the new.
alltce = tcenew.join(tceold, how='outer', on='tic_id', rsuffix='_old')
alltce = alltce.set_index('tic_id')

alltce = alltce.drop(columns=['row_id'])

def fillna(df, col_name):
    df.loc[df[col_name].isna(), col_name] = df.loc[df[col_name].isna(), col_name + '_old']

fillna(alltce, 'toi_id')
fillna(alltce, 'Disposition')
fillna(alltce, 'RA')
fillna(alltce, 'Dec')
fillna(alltce, 'Tmag')
fillna(alltce, 'Epoc')
fillna(alltce, 'Period')
fillna(alltce, 'Duration')
fillna(alltce, 'Transit_Depth')
fillna(alltce, 'Sectors')
fillna(alltce, 'camera')
fillna(alltce, 'ccd')
fillna(alltce, 'star_rad')
fillna(alltce, 'star_mass')
fillna(alltce, 'teff')
fillna(alltce, 'logg')
fillna(alltce, 'SN')
fillna(alltce, 'Qingress')

alltce = alltce.drop(columns=[c for c in alltce.columns if c.endswith('_old')])

alltce['Ilabel'] = alltce['Ilabel'].fillna(False)

# alltce.to_csv('../tce_bls_instar+old.csv')

In [ ]:
import os

from astroquery.mast import Catalogs
import numpy as np
import pandas as pd


include_liang_data = True


if include_liang_data:
    files = ['chdispositions.csv', 'avdispositions.csv', 'yldispositions.csv']
    tces_file = '../tce_bls_instar+old.csv'

else:
    files = ['chdispositions.csv', 'avdispositions.csv']
    tces_file = '../tce_bls_instar.csv'
    
ext_data_file = '../ext_mast_data.csv'


tce_table = pd.read_csv(tces_file, header=0).set_index('tic_id')
tce_table = tce_table.drop(columns=['Unnamed: 0'])
joined_table = tce_table

ext_table = pd.read_csv(ext_data_file, header=0).set_index('tic_id')
joined_table = joined_table.join(ext_table, on='tic_id', how='left')

joined_table = joined_table[
    joined_table["objType"].isnull()
    | (joined_table["objType"] == 'STAR')
]


joined_table['disp_E'] = 0
joined_table['disp_J'] = 0
joined_table['disp_N'] = 0
joined_table['disp_S'] = 0
joined_table['disp_B'] = 0

for name in files:
    with open(os.path.join('..', name)) as f:
        labels_table = pd.read_csv(f, header=0, usecols=[0, 1, 2])
        labels_table = labels_table.rename(columns={'tic': 'tic_id'}).set_index('tic_id')
        tag = name[:2]
        label = 'flag_' + tag
        labels_table = labels_table.rename(columns={'user': 'user_' + tag, 'flag': label})
        joined_table = joined_table.join(labels_table, on='tic_id', how='left')
        
        joined_table['disp_E'] += (joined_table[label] == 'E').map({True: 1, False: 0})
        joined_table['disp_N'] += (joined_table[label] == 'N').map({True: 1, False: 0})
        joined_table['disp_J'] += (joined_table[label] == 'J').map({True: 1, False: 0})
        joined_table['disp_B'] += (joined_table[label] == 'B').map({True: 1, False: 0})

        joined_table.loc[joined_table['Ilabel'], 'disp_E'] = 0
        joined_table.loc[joined_table['Ilabel'], 'disp_N'] = 0
        joined_table.loc[joined_table['Ilabel'], 'disp_J'] = 0
        joined_table.loc[joined_table['Ilabel'], 'disp_S'] = 1
        joined_table.loc[joined_table['Ilabel'], 'disp_B'] = 0

fltr = (~joined_table["flag_ch"].isnull()
        | ~joined_table["flag_av"].isnull())
    
if include_liang_data:
    fltr = fltr | ~joined_table["flag_yl"].isnull()
    
joined_table = joined_table[fltr]

if include_liang_data:
    joined_table.to_csv('astronet/tces-new+old.csv')
else:
    joined_table.to_csv('astronet/tces-new.csv')

```
python astronet/data/generate_input_records.py --input_tce_csv_file=astronet/tces-new.csv --tess_data_dir=/home/${USER}/lc --output_dir=astronet/tfrecords-new --num_worker_processes=8 --make_test_set
```

```
python astronet/data/generate_input_records.py --input_tce_csv_file=astronet/tces-new+old.csv --tess_data_dir=/home/${USER}/lc --output_dir=astronet/tfrecords-new+old --num_worker_processes=8 --make_test_set
```